In [1]:
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, BartForConditionalGeneration, BartTokenizer
import gradio as gr
import torch

c:\Users\Ahmad\anaconda3\envs\transformergpux\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("gpu: ", torch.cuda.get_device_name())
else:
    device = torch.device("cpu")
    print("No GPU available, using CPU.")

gpu:  NVIDIA GeForce RTX 2080


we will be using the gpu, we can switch to cpu or colab them together

In [3]:
ds = load_dataset("cnn_dailymail","1.0.0")

In [4]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})


some of the articles are too long for the model, we will trancuate to max number of tokens for Bart "1024"

In [5]:
train_ds = ds['train']
print(len(train_ds['article'][6]))

4819


In [16]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
model.to(device)
def Summarizer(text):
    inputs = tokenizer(text,return_tensors='pt',max_length=1024,truncation=True)
    inputs.to(device)
    summary_ids = model.generate(inputs["input_ids"])
    summary = tokenizer.decode(summary_ids[0],skip_special_tokens=True)
    return summary


In [19]:
train_ds = ds['train']
for index in range(7):
    test = train_ds[index]
    test = Summarizer(test['article'])
    print(test)

Harry Potter star Daniel Radcliffe turns 18 on Monday. He gains access to a reported £20 million ($41.1 million) fortune. Radcliffe's earnings from the first five Potter films have been held in a trust fund. Details of how he'll mark his landmark birthday are under wraps.
Judge Steven Leifman is an advocate for justice and the mentally ill. About one-third of all people in Miami-Dade county jails are mentally ill, he says. He says the sheer volume is overwhelming the system. Starting in 2008, many inmates will be sent to a new mental health facility.
NEW: "I probably had a 30-, 35-foot free fall," survivor Gary Babineau says. NEW: "My truck was completely face down, pointed toward the ground, and my truck got ripped in half," he says. Emergency room physician John Hink rushed to the scene in 15 minutes. Rescue effort was controlled and organized, he says; opposite of the lightning-quick collapse.
Doctors remove five small polyps from President Bush's colon. All were small, less than a 

that works, alright,

In [21]:
with gr.Blocks() as demo:
  with gr.Row():
    uInput = gr.Textbox()
  with gr.Row():
    run = gr.Button("Summarize")
    output = gr.Textbox(label="Summarized")

  event = run.click(
      fn = Summarizer,
      inputs = [uInput],
      outputs =[output]
  )

In [23]:
demo.launch(share=True)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Running on public URL: https://796a11757ac59193f0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
def training_tokenizer(dataset):
    inputs = dataset['article']
    targts = dataset['highlights']
    model_inputs = tokenizer(inputs,max_length=1024,truncation=True)
    model_inputs.to(device)